In [1]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
with_mask = with_mask.reshape(200,50 * 50 * 3)

In [4]:
without_mask = without_mask.reshape(200,50 * 50 * 3)

In [5]:
X = np.r_[with_mask, without_mask]

In [6]:
y = np.zeros(X.shape[0])

In [7]:
y[200:] = 1.0

In [8]:
names = {0.0: 'Mask', 1 : 'No Mask'}

In [9]:
# SVM - Support Vector Machine
# SVC - Support Vector Classification
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [11]:
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [13]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [14]:
#x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

In [15]:
accuracy_score(y_test, y_pred)

1.0

In [16]:
haar_data = cv2.CascadeClassifier('haar_cascade/haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    img = cv2.flip(img,1)
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1, -1)
            #face = pca.transform(face)
            pred = svm.predict(face)
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font, 1, (244,250,250), 4)
            #print(n)
        cv2.imshow('result', img) # Parameters (<name of the window that will open>, <variable_name>)
        if cv2.waitKey(2) == 27: # 27 is ASCII value of Escape
            break
capture.release()
cv2.destroyAllWindows()